In [ ]:
pip install paho-mqtt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Jacob -csv code:

import os
import csv

# Function to read all CSV files in a folder
def read_csvs_in_folder(folder_path):
    all_rows = []

    # List all files in the folder
    file_list = os.listdir(folder_path)

    # Filter only CSV files
    csv_files = [file for file in file_list if file.endswith('.csv')]

    # Read each CSV file
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            rows = list(csv_reader)
            all_rows.extend(rows)

    return all_rows

# Modify the existing script to use the read_csvs_in_folder function
orders_folder_path = 'order_csvs'
csv_rows_in_folder = read_csvs_in_folder(folder_path)


# Close all open CSV files
for csv_file, _ in csv_files.values():
    csv_file.close()

In [ ]:
import paho.mqtt.client as mqtt
import tkinter as tk
from tkinter import ttk
import csv
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Dictionary to store CSV file writers and file objects for each topic
csv_files = {}

# Function to be called whenever a new message is received on a subscribed MQTT topic
def on_message(client, userdata, message):
    topic = message.topic
    payload = message.payload.decode()

    # Update the GUI with the most recent value for the subscribed topic
    if topic in topic_labels:
        topic_labels[topic].config(text=f"{topic}: {payload}")

    # Store the received value in the corresponding CSV file
    if topic in csv_files:
        csv_file, csv_writer = csv_files[topic]
        csv_writer.writerow([payload])

# Function to publish motor voltage to the broker
def publish_voltage():
    voltage = voltage_entry.get()
    topic = "Conveyor04/set_speed"
    client.publish(topic, voltage)

# Function to publish run status to the broker (0 or 1)
def publish_run_status():
    run_status = run_status_var.get()
    topic = "Conveyor04/set_run_status"
    client.publish(topic, str(run_status))

# Function to exit the application
def exit_application():
    # Close all open CSV files
    for csv_file, _ in csv_files.values():
        csv_file.close()
    root.destroy()

# Create an instance of the MQTT client
client = mqtt.Client()

# Set the on_message attribute of the client to the function on_message
client.on_message = on_message

# Define the MQTT broker's address and port
broker_address = "129.97.228.106"
broker_port = 1883

# Connect to the MQTT broker
client.connect(broker_address, broker_port)

# Subscribe to multiple relevant topics
topics = ["Conveyor04/get_belt_speed", "Conveyor03/box_shape", "Conveyor03/box_enc"]
for topic in topics:
    client.subscribe(topic)

# Create and configure the GUI
root = tk.Tk()
root.title("MQTT Data Monitoring")

# Maximize the main window to fit the full screen
root.attributes("-fullscreen", True)

# Create a custom style with the desired font size
custom_style = ttk.Style()
custom_style.configure('Custom.TLabel', font=("Helvetica", 18))

# Create labels to display the most recent values for subscribed topics with the custom style
topic_labels = {}
for i, topic in enumerate(topics):
    topic_label = ttk.Label(root, text=f"{topic}:", style='Custom.TLabel')
    topic_label.grid(row=i, column=0, padx=10, pady=10, sticky='w')
    topic_labels[topic] = topic_label

# Create an entry box to input motor voltage with the custom style
voltage_label = ttk.Label(root, text="Enter Motor Voltage:", style='Custom.TLabel')
voltage_label.grid(row=len(topics), column=0, padx=10, pady=10, sticky='w')
voltage_entry = ttk.Entry(root, font=("Helvetica", 18))
voltage_entry.grid(row=len(topics), column=1, padx=10, pady=10, sticky='w')
voltage_button = ttk.Button(root, text="Publish Voltage", command=publish_voltage, style='Custom.TButton')
voltage_button.grid(row=len(topics), column=2, padx=10, pady=10, sticky='w')

# Create a Checkbutton to control the conveyor's run status with the custom style
run_status_label = ttk.Label(root, text="Conveyor Run Status:", style='Custom.TLabel')
run_status_label.grid(row=len(topics) + 1, column=0, padx=10, pady=10, sticky='w')
run_status_var = tk.IntVar()
run_status_switch = ttk.Checkbutton(root, text="Run", variable=run_status_var, command=publish_run_status, style='Custom.TCheckbutton')
run_status_switch.grid(row=len(topics) + 1, column=1, padx=10, pady=10, sticky='w')

# Create a function to update the live graph
def update_graph(x_data, y_data, line, ax):
    try:
        # Get the new y-value
        new_value = float(topic_labels["Conveyor04/get_belt_speed"].cget("text").split(': ')[1])

        # Update x and y data with new values
        x_data.append(len(x_data))
        y_data.append(new_value)

        # Ensure that x_data and y_data have the same length
        if len(x_data) > len(y_data):
            x_data.pop(0)
        elif len(y_data) > len(x_data):
            y_data.pop(0)

        # Update the graph
        line.set_data(x_data, y_data)
        ax.relim()
        ax.autoscale_view()
        canvas.draw()
    except Exception as e:
        print(e)

# Create a figure for the live graph
fig = Figure(figsize=(6, 4), dpi=100)
ax = fig.add_subplot(111)
x_data = []
y_data = []
line, = ax.plot(x_data, y_data)

# Create a canvas to display the live graph in the GUI
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.grid(row=len(topics) + 3, column=0, columnspan=3, padx=10, pady=10)

# Add a label for the live graph
live_graph_label = ttk.Label(root, text="Live Belt Speed Graph:", style='Custom.TLabel')
live_graph_label.grid(row=len(topics) + 2, column=0, columnspan=3, padx=10, pady=10)

# Schedule the update_graph function to be called periodically
def update_graph_periodically():
    update_graph(x_data, y_data, line, ax)
    root.after(1000, update_graph_periodically)  # Update every 1000 milliseconds (1 second)

# Start the update_graph function to continuously update the graph
update_graph_periodically()

# Create an "Exit" button with the custom style
exit_button = ttk.Button(root, text="Exit", command=exit_application, style='Custom.TButton')
exit_button.grid(row=len(topics) + 2, column=6, padx=10, pady=20, sticky='se')

# Open CSV files for writing (assuming they exist)
for topic in topics:
    valid_topic = topic.replace('/', '_')
    csv_file = open(f"{valid_topic}_data.csv", "a", newline="")
    csv_writer = csv.writer(csv_file)
    csv_files[topic] = (csv_file, csv_writer)

# Start the MQTT client's main loop to wait for updates on the subscribed topics
client.loop_start()

# Keep the GUI running
root.mainloop()

# Close all open CSV files
for csv_file, _ in csv_files.values():
    csv_file.close()

# Stop the MQTT client's main loop and disconnect from the broker
client.loop_stop()
client.disconnect()

TimeoutError: timed out

In [ ]:
import os
import csv
import glob
import shutil


def read_csv(file_path):
    """
    Opens and reads the rows of a CSV file.

    Parameters:
    - file_path (str): The path to the CSV file.

    Returns:
    - list of lists: A list where each element is a list representing a row in the CSV file.
    """
    rows = []

    with open(file_path, 'r') as csvfile:
        # Create a CSV reader object
        csv_reader = csv.reader(csvfile)

        # Read each row and append it to the list
        for row in csv_reader:
            rows.append(row)

    
    return rows




def move_csv_file(source_path, destination_folder):
    """
    Moves a CSV file from the source path to the specified destination folder.

    Parameters:
    - source_path (str): The path to the CSV file to be moved.
    - destination_folder (str): The path to the destination folder.
    
    Returns:
    - str or None: The new path of the moved CSV file, or None if the operation failed.
    """
    try:
        # Ensure the destination folder exists
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # Extract the filename from the source path
        file_name = os.path.basename(source_path)

        # Create the new path in the destination folder
        destination_path = os.path.join(destination_folder, file_name)

        # Move the file
        shutil.move(source_path, destination_path)

        return destination_path
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage:
source_csv_path = 'path/to/your/source/file.csv'
destination_folder = 'path/to/your/destination/folder'


def set_csv_row_value(file_path, column_name, new_value):
    """
    Sets the value of a specific column in a CSV file to a specified value.

    Parameters:
    - file_path (str): The path to the CSV file.
    - column_name (str): The name of the column to be modified.
    - new_value: The new value to be set in the specified column.

    Returns:
    - bool: True if the modification was successful, False otherwise.
    """
    try:
        # Read the existing content
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            rows = list(csv_reader)

        # Find and set the value in the specified column
        column_index = None
        for i, header in enumerate(rows[0]):
            if header == column_name:
                column_index = i
                break

        if column_index is not None:
            for row in rows[1:]:
                row[column_index] = str(new_value)

            # Write the modified content back to the file
            with open(file_path, 'w', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerows(rows)

            return True
        else:
            print(f"Column '{column_name}' not found.")
            return False
    except Exception as e:
        print(f"Error: {e}")
        return False


    

In [ ]:
#Data Cell

import tkinter as tk
from tkinter import ttk

O_I_P_path = "order_csvs\Orders_in_progress"

shapes = ['Circle','Pentagon','Trapezoid', 'Triangle']
###########################################################################Values to be updated

# The data flow is as follows, on start-up the factory status file should be opened, and filled with the relevant info from node red, and the orders in progress tab.  When the next-order button is pressed,
# the next csv file is opened, and the order is read in, need to move the file to in orders_in_progress folder, which has to have extra rows filled (first data row) for which conveyor, and how many is filled in.
# (need a function for this process). Then this can be continually updated until the exit button is pressed, where it is simply closed, or when it is completed, where the order can be moved to the Completed_orders 
# folder, also with more info on what order number it is (which has to be the first row of the csv) 

# Next structure created by jacob, with plc order 3,4,8,9 for intermediate data. conveyor_status_obj[plc_value] has indexes 'filepath' 'order_number' 'complete_status'
conveyor_status_obj = []

Conveyor03 = []
Conveyor04 = []
Conveyor08 = []
Conveyor09 = []


#Need to be attained from plc
completed_orders = 1
error_status1 = True
error_status2 = False
box_stuck_error=True
count_error=True

#Logic to attain order complete status, 1 = complete, 0 = incomplete
Conveyor03_complete = 1
Conveyor04_complete = 0
Conveyor08_complete = 1
Conveyor09_complete = 0



#Array incremented through sensor/actuation and through the csv file on startup
Conveyor03_collected = [1,1,0,0]
Conveyor04_collected = [1,2,1,0]
Conveyor08_collected = [1,2,0,0]
Conveyor09_collected = [1,0,0,0]

#Need to be updated through next csv file
Conveyor03.extend([1, 2, 2, 4])
Conveyor04.extend([1, 2, 2, 4])
Conveyor08.extend([4, 1, 3, 4])
Conveyor09.extend([1, 2, 3, 4])

##When increment completed orders is called, we need to read new information into the respective conveyor, Add the reading and storing function of CSV files
# into each of the increment completed order functions


#########################################################################

# makes a dictionary from the csv rows
def populate_array_from_csv(file_path):
    """
    Populates an array with indexes from the first column and values from the second column of a CSV file.

    Parameters:
    - file_path (str): The path to the CSV file.

    Returns:
    - dict: A dictionary where keys are indexes (from the first column) and values are corresponding values (from the second column).
    """
    result_dict = {}

    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        
        for row in csv_reader:
            # Assuming the CSV has at least two columns
            if len(row) >= 2:
                index, value = row[0], row[1]
                result_dict[index] = value

    return result_dict

#this function used on startup and new order reading.
def filled_desired_shapes_from_csv_row(row, plc_value):

    wanted_list = []
    wanted_list[0] = row['circle'] # Circle
    wanted_list[1] = row['pentagon'] # Pentagon
    wanted_list[2] = row['trapeziod'] # Trapezoid
    wanted_list[3] = row['triangle']  # Triangle

    if(plc_value == 3):
        Conveyor03_complete = 0
        Conveyor03 = wanted_list
    elif(plc_value == 4):
        Conveyor04_complete = 0
        Conveyor04 = wanted_list
    elif(plc_value == 8):
        Conveyor08_complete = 0
        Conveyor08 = wanted_list
    elif(plc_value == 9):
        Conveyor09_complete = 0
        Conveyor09 = wanted_list

    return


#limitiation is that if there were somehow more than one open order, that order would likely get lost
def read_order_csv(file_path):
    """
    Reads a CSV file, appends the data to the given conveyor list,
    and populates the corresponding collected array based on the PLC row.

    Parameters:
    - file_path (str): The path to the CSV file.
    """
    rows_dic = populate_array_from_csv(file_path)
    
    collected_list = []
    collected_list[0] = rows_dic['circle_collected']  # Circle
    collected_list[1] = rows_dic['pentagon_collected'] # Pentagon
    collected_list[2] = rows_dic['trapeziod_collected']  # Trapezoid
    collected_list[3] = rows_dic['triangle_collected']  # Triangle

    wanted_list = []
    wanted_list[0] = rows_dic['circle']  # Circle
    wanted_list[1] = rows_dic['pentagon'] # Pentagon
    wanted_list[2] = rows_dic['trapeziod']  # Trapezoid
    wanted_list[3] = rows_dic['triangle']  # Triangle

    order_complete = (wanted_list == collected_list)  

    plc_value = rows_dic['plc']
    
    # Below are what could be swapped out for not hardcoded values
    conveyor_status_obj[plc_value]["order_number"] = rows_dic['order_num']
    conveyor_status_obj[plc_value]["complete_status"] = order_complete
    conveyor_status_obj[plc_value]["file_path"] = file_path


    if(plc_value == 3):
        Conveyor03_collected = collected_list
        Conveyor03 = wanted_list
        Conveyor03_complete = order_complete
    elif(plc_value == 4):
        Conveyor04_collected = collected_list
        Conveyor04 = wanted_list
        Conveyor04_complete = order_complete
    elif(plc_value == 8):
        Conveyor08_collected = collected_list
        Conveyor08 = wanted_list
        Conveyor08_complete = order_complete
    elif(plc_value == 9):
        Conveyor09_collected = collected_list
        Conveyor09 = wanted_list
        Conveyor09_complete = order_complete
    return
    
#called when the ui first starts, populates the above arrays
def startup_ui_csv_files():
    for filename in os.listdir(O_I_P_path):
        if filename.endswith('.csv'):
            file_path = os.path.join("Orders_in_progress", filename)
            read_order_csv(file_path)

# Function to read the next order csv, needs to move the file to the next folder,
def fetch_next_order(plc):
    """
    Get the oldest CSV file from a folder based on creation time.

    Parameters:
    - folder_path (str): The path to the folder.

    Returns:
    - str or None: The path to the oldest CSV file, or None if no CSV files are found.
    """
    csv_files = glob.glob(os.path.join(orders_folder_path, '*.csv'))

    if not csv_files:
        return None

    oldest_csv_file = min(csv_files, key=os.path.getctime)
    
    rows = populate_array_from_csv(oldest_csv_file)

    if set_csv_row_value(oldest_csv_file, "plc", plc):
        print(f"Column plc in '{oldest_csv_file}' set to {plc} successfully.")
        # Don't need to update order complete, as it will be done by the update_order_completion_status() function.
    else:
        print(f"Failed to set column plc in '{oldest_csv_file}'.")
    
    filled_desired_shapes_from_csv_row(rows, rows['plc'])
    
    new_path = move_csv_file(oldest_csv_file, "Orders_in_progress")

    if new_path:
        conveyor_status_obj[plc]["filepath"] = new_path
        print(f"CSV file moved to: {new_path}")
    else:
        print("Failed to move the CSV file.")
    
    

# function for when order is completed, and needs to be moved to competed, and status from it taken. Then gets processed
def order_complete(plc):
    #TODO make the summarizing csv
    folder_path = conveyor_status_obj[plc]["filepath"]
    new_path = move_csv_file(folder_path, "Completed_orders")
    fetch_next_order(plc)
    
    
#
def update_order_completion_status():
    if(Conveyor03_collected == Conveyor03):
        completed_orders = completed_orders + 1
        Conveyor03_complete = 1
        conveyor_status_obj[3]["complete_status"]
        order_complete(3)
    elif(Conveyor04_collected == Conveyor04):
        completed_orders = completed_orders + 1
        Conveyor04_complete = 1
        conveyor_status_obj[4]["complete_status"]
        order_complete(4)
    elif(Conveyor08_collected == Conveyor08):
        completed_orders = completed_orders + 1
        Conveyor08_complete = 1
        conveyor_status_obj[8]["complete_status"]
        order_complete(8)
    elif(Conveyor09_collected == Conveyor09):
        completed_orders = completed_orders + 1
        Conveyor09_complete = 1
        conveyor_status_obj[9]["complete_status"]
        order_complete(9)



IndentationError: expected an indented block after function definition on line 186 (4281340092.py, line 191)

: 

In [ ]:
#HMI cell

# Create new arrays to store the difference between conveyor and collected values//No need to be updated
Conveyor09_left = [c - collected for c, collected in zip(Conveyor09, Conveyor09_collected)]
Conveyor08_left = [c - collected for c, collected in zip(Conveyor08, Conveyor08_collected)]
Conveyor03_left = [c - collected for c, collected in zip(Conveyor03, Conveyor03_collected)]
Conveyor04_left = [c - collected for c, collected in zip(Conveyor04, Conveyor04_collected)]

def increment_completed_orders():
    global completed_orders
    completed_orders += 1
    update_completed_orders_label()


def update_completed_orders_label():
    completed_orders_label_1.config(text=f"Completed Orders: {completed_orders}")

def update_error_label():
    error_label.config(text="Error")
    update_error_status1()
    update_error_status2()
    update_error_status3()
    update_error_status4()

def update_error_status1():
    if error_status1:
        error_status_label.config(bg="red")  # Setting background color to red for error
    else:
        error_status_label.config(bg="green")  # Setting background color to green for no error

def update_error_status2():
    if error_status2:
        error_status2_label.config(bg="orange")  # Setting background color to red for error
    else:
        error_status2_label.config(bg="green")  # Setting background color to green for no error

def update_error_status3():
    global box_stuck_error
    if box_stuck_error:
        error_status3_label.config(bg="red")  # Setting background color to red for ball stuck error
    else:
        error_status3_label.config(bg="green")  # Setting background color to green when no ball stuck error

def update_error_status4():
    global count_error
    if count_error:
        error_status4_label.config(bg="red")  # Setting background color to red for ball stuck error
    else:
        error_status4_label.config(bg="green")  # Setting background color to green when no ball stuck error

def update_conveyor03_status():
    if Conveyor03_complete == 1:
        completion_light_label_03.config(bg="green")  # Turn LED to green
        completion_light_label_032.config(bg="green")  # Turn LED to green
    else:
        completion_light_label_03.config(bg="red")  # Turn LED to red
        completion_light_label_032.config(bg="red")  # Turn LED to red

def update_conveyor04_status():
    if Conveyor04_complete == 1:
        completion_light_label_04.config(bg="green")  # Turn LED to green
        completion_light_label_042.config(bg="green")  # Turn LED to green
    else:
        completion_light_label_04.config(bg="red")  # Turn LED to red
        completion_light_label_042.config(bg="red")  # Turn LED to red

def update_conveyor08_status():
    if Conveyor08_complete == 1:
        completion_light_label_08.config(bg="green")  # Turn LED to green
        completion_light_label_082.config(bg="green")  # Turn LED to green
    else:
        completion_light_label_08.config(bg="red")  # Turn LED to red
        completion_light_label_082.config(bg="red")  # Turn LED to red

def update_conveyor09_status():
    if Conveyor09_complete == 1:
        completion_light_label_09.config(bg="green")  # Turn LED to green
        completion_light_label_092.config(bg="green")  # Turn LED to green
    else:
        completion_light_label_09.config(bg="red")  # Turn LED to red
        completion_light_label_092.config(bg="red")  # Turn LED to red

def update_conveyor_labels():
    conveyor03_status_label.config(text=f"Order: {Conveyor03}, Completed: {Conveyor03_collected}")
    conveyor03_left_label.config(text=f" Order Left: {Conveyor03_left}")

    conveyor04_status_label.config(text=f"Order: {Conveyor04}, Completed: {Conveyor04_collected}")
    conveyor04_left_label.config(text=f" Order Left: {Conveyor04_left}")

    conveyor08_status_label.config(text=f"Order: {Conveyor08}, Completed: {Conveyor08_collected}")
    conveyor08_left_label.config(text=f" Order Left: {Conveyor08_left}")

    conveyor09_status_label.config(text=f"Order: {Conveyor09}, Completed: {Conveyor09_collected}")
    conveyor09_left_label.config(text=f" Order Left: {Conveyor09_left}")

# TODO need to write function calls for the new order status here for these. 
#note from Jacob, moved the order complete increment to the update_order_completion_status function
def increment_completed_orders_03():
    global completed_orders, Conveyor03, Conveyor03_collected, Conveyor03_left
    Conveyor03.clear()
    Conveyor03_collected = [0, 0, 0, 0]
    fetch_next_order(3)
    Conveyor03_left = [c - collected for c, collected in zip(Conveyor03, Conveyor03_collected)]
    update()


def increment_completed_orders_04():
    global completed_orders, Conveyor04, Conveyor04_collected, Conveyor04_left

    Conveyor04.clear()
    Conveyor04_collected = [0, 0, 0, 0]
    fetch_next_order(4)
    Conveyor04_left = [c - collected for c, collected in zip(Conveyor04, Conveyor04_collected)]
    update()


def increment_completed_orders_08():
    global completed_orders, Conveyor08, Conveyor08_collected, Conveyor08_left

    Conveyor08.clear()
    Conveyor08_collected = [0, 0, 0, 0]
    fetch_next_order(8)
    Conveyor08_left = [c - collected for c, collected in zip(Conveyor08, Conveyor08_collected)]
    update()


def increment_completed_orders_09():
    global completed_orders, Conveyor09, Conveyor09_collected, Conveyor09_left
    Conveyor09.clear()
    Conveyor09_collected = [0, 0, 0, 0]
    fetch_next_order(9)
    Conveyor09_left = [c - collected for c, collected in zip(Conveyor09, Conveyor09_collected)]
    update()

def update():
    update_order_completion_status()
    update_error_label()
    update_conveyor_labels()
    update_completed_orders_label()
    update_conveyor03_status()
    update_conveyor04_status()
    update_conveyor08_status()
    update_conveyor09_status()


root = tk.Tk()
root.title("Conveyor Orders")

tab_control = ttk.Notebook(root)

# Tab 1 - Home
tab1 = ttk.Frame(tab_control)
tab_control.add(tab1, text='Home')
tab_control.pack(expand=1, fill="both")

tab2 = ttk.Frame(tab_control)
tab_control.add(tab2, text='Orders')
tab_control.pack(expand=1, fill="both")

# Creating a frame within the Home tab
home_frame = ttk.Frame(tab1)
home_frame.pack(expand=1, fill="both")

home_frame2 = ttk.Frame(tab2)
home_frame2.pack(expand=1, fill="both")


# Label to display completed orders
completed_orders_label_1 = tk.Label(home_frame, text=f"Total Completed orders: {completed_orders}", font=("Arial", 14))
completed_orders_label_1.grid(row=9, column=0, pady=90)  # Adjust row and pady for position


# Error labels
error_label = tk.Label(home_frame, text="Error", font=("Arial", 14))
error_label.grid(row=1, column=0)  # Adjust row and pady for position

error_label1 = tk.Label(home_frame, text="E-STOP", font=("Arial", 14))
error_label1.grid(row=2, column=0)  # Adjust row and pady for position

error_label2 = tk.Label(home_frame, text="Speed Error", font=("Arial", 14))
error_label2.grid(row=3, column=0)  # Adjust row and pady for position

error_label3 = tk.Label(home_frame, text="Box Stuck Error", font=("Arial", 14))
error_label3.grid(row=4, column=0)  # Adjust row and column for position

error_label4 = tk.Label(home_frame, text="Count Error", font=("Arial", 14))
error_label4.grid(row=5, column=0)  # Adjust row and column for position

# Label for representing error status with colored lights
error_status_label = tk.Label(home_frame, width=1, height=1, bg="red")  # Initial status is error (red)
error_status_label.grid(row=2, column=1, padx=5)  # Adjust row and padx for position

error_status2_label = tk.Label(home_frame, width=1, height=1, bg="red")  # Initial status is error (red)
error_status2_label.grid(row=3, column=1, padx=5)  # Adjust row and padx for position

error_status3_label = tk.Label(home_frame, width=1, height=1, bg="green")  # Initial status is no error (green)
error_status3_label.grid(row=4, column=1, padx=5)  # Adjust row and column for position

error_status4_label = tk.Label(home_frame, width=1, height=1, bg="green")  # Initial status is no error (green)
error_status4_label.grid(row=5, column=1, padx=5)  # Adjust row and column for position

# Label for Running Status of Conveyor03
running_status_label_03 = tk.Label(home_frame, text="Conveyor03: Complete", font=("Arial", 14))
running_status_label_03.grid(row=1, column=2, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor03
completion_light_label_03 = tk.Label(home_frame, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_03.grid(row=1, column=3, padx=5)  # Adjust row and padx for position

shapes_label = tk.Label(home_frame, text=f"Shapes order:{shapes}", font=("Arial", 12))
shapes_label.grid(row=0, column=2, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

# Label for displaying Conveyor03 array completion status
conveyor03_status_label = tk.Label(home_frame, text=f"Conveyor03:{Conveyor03}", font=("Arial", 12))
conveyor03_status_label.grid(row=2, column=2, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

# Label for Running Status of Conveyor04
running_status_label_04 = tk.Label(home_frame, text="Conveyor04: Complete", font=("Arial", 14))
running_status_label_04.grid(row=3, column=2, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor04
completion_light_label_04 = tk.Label(home_frame, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_04.grid(row=3, column=3, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor04 array completion status
conveyor04_status_label = tk.Label(home_frame, text=f"Conveyor04: {Conveyor04}", font=("Arial", 12))
conveyor04_status_label.grid(row=4, column=2, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

# Label for Running Status of Conveyor08
running_status_label_08 = tk.Label(home_frame, text="Conveyor08: Complete", font=("Arial", 14))
running_status_label_08.grid(row=5, column=2, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor08
completion_light_label_08 = tk.Label(home_frame, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_08.grid(row=5, column=3, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor08 array completion status
conveyor08_status_label = tk.Label(home_frame, text=f"Conveyor08: {Conveyor08}", font=("Arial", 12))
conveyor08_status_label.grid(row=6, column=2, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

# Label for Running Status of Conveyor09
running_status_label_09 = tk.Label(home_frame, text="Conveyor09: Complete", font=("Arial", 14))
running_status_label_09.grid(row=7, column=2, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor09
completion_light_label_09 = tk.Label(home_frame, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_09.grid(row=7, column=3, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor09 array completion status
conveyor09_status_label = tk.Label(home_frame, text=f"Conveyor09: {Conveyor09}", font=("Arial", 12))
conveyor09_status_label.grid(row=8, column=2, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

########Orders tab##################################################################################################
shapes_label = tk.Label(home_frame2, text=f"Shapes order:{shapes}", font=("Arial", 12))
shapes_label.grid(row=1, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

# Label to display completed orders
completed_orders_label_1 = tk.Label(home_frame2, text=f"Total Completed orders: {completed_orders}", font=("Arial", 14))
completed_orders_label_1.grid(row=0, column=0, pady=0)  # Adjust row and pady for position

# Label for Running Status of Conveyor03
running_status_label_03 = tk.Label(home_frame2, text="Conveyor03 Order Complete:", font=("Arial", 14))
running_status_label_03.grid(row=2, column=0, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor03
completion_light_label_032 = tk.Label(home_frame2, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_032.grid(row=2, column=1, padx=5)  # Adjust row and padx for position


# Label for displaying Conveyor03 array completion status
conveyor03_status_label = tk.Label(home_frame2,
                                     text=f"Order: {Conveyor03}, Completed: {Conveyor03_collected}",
                                     font=("Arial", 12))
conveyor03_status_label.grid(row=3, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

conveyor03_left_label = tk.Label(home_frame2,
                                     text=f" Order Left: {Conveyor03_left}",
                                     font=("Arial", 12))
conveyor03_left_label.grid(row=4, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position


# Label for Running Status of Conveyor04
running_status_label_04 = tk.Label(home_frame2, text="Conveyor04 Order Complete", font=("Arial", 14))
running_status_label_04.grid(row=5, column=0, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor04
completion_light_label_042 = tk.Label(home_frame2, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_042.grid(row=5, column=1, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor04 array completion status
conveyor04_status_label = tk.Label(home_frame2, text=f"Conveyor04: {Conveyor04}, Completed: {Conveyor04_collected}", font=("Arial", 12))
conveyor04_status_label.grid(row=6, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

conveyor04_left_label = tk.Label(home_frame2,
                                     text=f" Order Left: {Conveyor04_left}",
                                     font=("Arial", 12))
conveyor04_left_label.grid(row=7, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position


# Label for Running Status of Conveyor08
running_status_label_08 = tk.Label(home_frame2, text="Conveyor08 Order Complete", font=("Arial", 14))
running_status_label_08.grid(row=8, column=0, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor08
completion_light_label_082 = tk.Label(home_frame2, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_082.grid(row=8, column=1, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor08 array completion status
conveyor08_status_label = tk.Label(home_frame2, text=f"Conveyor08: {Conveyor08}, Completed: {Conveyor08_collected}", font=("Arial", 12))
conveyor08_status_label.grid(row=9, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

conveyor08_left_label = tk.Label(home_frame2,
                                     text=f" Order Left: {Conveyor08_left}",
                                     font=("Arial", 12))
conveyor08_left_label.grid(row=10, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position


# Label for Running Status of Conveyor09
running_status_label_09 = tk.Label(home_frame2, text="Conveyor09 Order Complete", font=("Arial", 14))
running_status_label_09.grid(row=11, column=0, pady=10, padx=20)  # Adjust row, column, pady, and padx for position

# Label for completion light indicator for Conveyor09
completion_light_label_092 = tk.Label(home_frame2, width=1, height=1)  # Initial status is completion (green/red)
completion_light_label_092.grid(row=11, column=1, padx=5)  # Adjust row and padx for position

# Label for displaying Conveyor09 array completion status
conveyor09_status_label = tk.Label(home_frame2, text=f"Conveyor09: {Conveyor09}, Completed: {Conveyor09_collected}", font=("Arial", 12))
conveyor09_status_label.grid(row=12, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

conveyor09_left_label = tk.Label(home_frame2,
                                     text=f" Order Left: {Conveyor09_left}",
                                     font=("Arial", 12))
conveyor09_left_label.grid(row=13, column=0, columnspan=2, pady=5, padx=20)  # Adjust row, column, pady, and padx for position

update_conveyor03_button = tk.Button(home_frame2, text="Next Order", command=increment_completed_orders_03)
update_conveyor03_button.grid(row=2, column=2)

update_conveyor04_button = tk.Button(home_frame2, text="Next Order", command=increment_completed_orders_04)
update_conveyor04_button.grid(row=5, column=2)

update_conveyor08_button = tk.Button(home_frame2, text="Next Order", command=increment_completed_orders_08)
update_conveyor08_button.grid(row=8, column=2)

update_conveyor09_button = tk.Button(home_frame2, text="Next Order", command=increment_completed_orders_09)
update_conveyor09_button.grid(row=11, column=2)




# Function calls to update various conveyor completion statuses
update()

root.mainloop()